In [53]:
import pandas
import pickle
import ipecharts
import pydynamo
import uncertainties
import collections
import matplotlib
import matplotlib.pyplot as plt


base = "SW_eos"
# Load the files from the fetch_data
state_data = pickle.load(open(f"{base}.raw_data.pkl", "rb"))
state_vars = set([j[0] for k in state_data.keys() for j in k])

# Turn state variables into a multi-index, with a particular preferred order
preferred_state_order = ("Lambda", "kT", "ndensity", "N")
state_vars = sorted(state_vars, key=lambda x: preferred_state_order.index(x) if x in preferred_state_order else -1)
orig_df = pickle.load(open(f"{base}.df.pkl", "rb"))
orig_df.set_index(list(state_vars), inplace=True)

# Filter out columns that do not change (boring state variables i.e.)
#orig_df = orig_df.loc[:, orig_df.nunique() > 1]


# Create a dataframe of scalars
scalar_df = orig_df.copy()
for col in scalar_df.columns:
    if scalar_df[col].dtype == "object":
        if isinstance(scalar_df[col].iloc[0], collections.abc.Iterable):
            del scalar_df[col]
scalar_df

NEventsTot        tTotal  \
InitState Lambda kT  ndensity N                                
FCC       1.5    1.0 0.01     4000    40000000  23845.035932   
                     0.10     4000    40000000   1247.117008   
                     0.50     4000    40000000    682.734445   
                     1.00     4000    40000000    352.054365   
                     1.30     4000    40000000    153.405260   
                 1.5 0.01     4000    40000000  25590.701291   
                     0.10     4000    40000000   2671.554374   
                     0.50     4000    40000000    655.984167   
                     1.00     4000    40000000    299.920201   
                     1.30     4000    40000000    121.664559   
                 2.0 0.01     4000    40000000  25019.789374   
                     0.10     4000    40000000   2599.176014   
                     0.50     4000    40000000    574.301380   
                     1.00     4000    40000000    265.207320   
                     1.30     4000    40000000    103.538879   
                 2.5 0.01     4000    40000000  23962.158945   
                     0.10     4000    40000000   2460.516392   
                     0.50     4000    40000000    516.067796   
                     1.00     4000    40000000    240.088358   
                     1.30     4000    40000000     91.497844   
                 3.0 0.01     4000    40000000  22859.637892   
                     0.10     4000    40000000   2326.265672   
                     0.50     4000    40000000    471.629508   
                     1.00     4000    40000000    220.996613   
                     1.30     4000    40000000     82.885040   
                 inf 0.01     4000    40000000  48493.566977   
                     0.10     4000    40000000   4638.724670   
                     0.50     4000    40000000    814.186216   
                     1.00     4000    40000000    398.440755   
                     1.30     4000    40000000    136.303207   
          2.0    1.0 0.01     4000    40000000    461.927211   
                     0.10     4000    40000000    198.664502   
                     0.50     4000    40000000    143.602432   
                     1.00     4000    40000000    164.077613   
                     1.30     4000    40000000    150.882315   
                 1.5 0.01     4000    40000000  15156.357471   
                     0.10     4000    40000000    256.332924   
                     0.50     4000    40000000    173.414606   
                     1.00     4000    40000000    125.644637   
                     1.30     4000    40000000    124.828705   
                 2.0 0.01     4000    40000000  15303.873526   
                     0.10     4000    40000000    448.047739   
                     0.50     4000    40000000    269.417239   
                     1.00     4000    40000000    111.497971   
                     1.30     4000    40000000    108.680441   
                 2.5 0.01     4000    40000000  14762.070227   
                     0.10     4000    40000000   1410.947904   
                     0.50     4000    40000000    311.933787   
                     1.00     4000    40000000    101.762522   
                     1.30     4000    40000000     97.506480   
                 3.0 0.01     4000    40000000  14116.946908   
                     0.10     4000    40000000   1426.638658   
                     0.50     4000    40000000    290.486249   
                     1.00     4000    40000000     93.828259   
                     1.30     4000    40000000     89.105293   
                 inf 0.01     4000    40000000  29776.499814   
                     0.10     4000    40000000   2944.774958   
                     0.50     4000    40000000    532.284154   
                     1.00     4000    40000000    165.813940   
                     1.30     4000    40000000    155.369969   

                                                        p  \
InitSt

In [55]:
orig_df["ChungLu"]

InitState  Lambda  kT   ndensity  N   
FCC        1.5     1.0  0.01      4000    [(1, 1), (2, 2), (1, 2), (1, 3), (4, 5), (3, 5...
                        0.10      4000    [(8, 10), (10, 10), (6, 10), (10, 11), (7, 10)...
                        0.50      4000    [(8, 9), (8, 10), (8, 8), (8, 11), (8, 12), (4...
                        1.00      4000    [(13, 16), (13, 14), (13, 13), (12, 13), (14, ...
                        1.30      4000    [(18, 18), (17, 18), (17, 17), (16, 18), (16, ...
                   1.5  0.01      4000    [(1, 1), (2, 3), (1, 3), (1, 2), (2, 2), (2, 4...
                        0.10      4000    [(1, 2), (1, 1), (2, 4), (2, 3), (3, 3), (3, 5...
                        0.50      4000    [(6, 7), (6, 8), (5, 6), (6, 9), (7, 8), (8, 1...
                        1.00      4000    [(12, 14), (13, 14), (14, 14), (14, 15), (13, ...
                        1.30      4000    [(18, 18), (17, 18), (16, 18), (17, 17), (16, ...
                   2.0  0.01      4000   